In [1]:
! git clone "https://github.com/gdbwoo/DS-3001-Projects"

Cloning into 'DS-3001-Projects'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 56 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (56/56), 7.96 MiB | 5.36 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [18]:
# Import all packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [24]:
# Observe all unfiltered data
df = pd.read_csv('DS-3001-Projects/Project 2/fhs_train.csv')
print(df.shape, '\n')
df.head(50)

(3180, 17) 



,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1267,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,1209,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
2,2050,0,52,1.0,0,0.0,0.0,0,0,0,275.0,112.0,71.0,25.68,80,NaN,0
3,1183,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,3225,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0
5,1659,1,56,4.0,0,0.0,0.0,0,0,0,232.0,126.0,83.0,25.67,75,NaN,0
6,1054,0,54,1.0,0,0.0,0.0,0,1,0,237.0,171.5,105.5,34.25,91,104.0,0
7,3455,1,47,4.0,0,0.0,0.0,0,1,0,254.0,138.0,96.0,29.73,63,69.0,0
8,2387,1,40,1.0,1,20.0,0.0,0,0,0,332.0,114.0,76.0,26.51,70,90.0,0
9,3313,0,61,1.0,1,9.0,0.0,0,0,0,259.0,134.5,87.0,22.91,70,91.0,0


In [22]:
# Observe 50 data with positive TenYearCHD to look for any patterns or trends among those at risk of CHD
# df = df.sort_values('TenYearCHD', ascending=False)
# df.head(50)

In [25]:
# Remove all nan/missing values
df = df.dropna()
print(df.shape, '\n')
df.head(50)

(2744, 17) 



,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1267,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,1209,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
3,1183,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,3225,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0
6,1054,0,54,1.0,0,0.0,0.0,0,1,0,237.0,171.5,105.5,34.25,91,104.0,0
7,3455,1,47,4.0,0,0.0,0.0,0,1,0,254.0,138.0,96.0,29.73,63,69.0,0
8,2387,1,40,1.0,1,20.0,0.0,0,0,0,332.0,114.0,76.0,26.51,70,90.0,0
9,3313,0,61,1.0,1,9.0,0.0,0,0,0,259.0,134.5,87.0,22.91,70,91.0,0
10,1234,1,59,1.0,0,0.0,0.0,0,0,0,229.0,100.5,66.0,25.18,44,81.0,0
11,1185,0,64,2.0,1,3.0,0.0,0,0,0,315.0,135.0,80.0,25.23,103,89.0,0
